# Tarea 5. Modelo de Mezclas Gaussianas.

<img style="float: right; margin: 0px 0px 15px 15px;" src="https://upload.wikimedia.org/wikipedia/commons/thumb/1/1f/Gauss-sum.svg/800px-Gauss-sum.svg.png" width="500px" height="300px" />

___

[0.09866961 0.70947177 0.19185862]


## 1. 

Programar el modelo de mezclas Gaussianas en una función.

### 1. Inicialización aleatoria de los parámetros: 

   - $\pi_c^0$ se puede inicializar con una muestra de la distribución de Dirichlet.
   - $\mu_c^0$ se puede inicializar eligiendo aleatoriamente datos de la muestra para entrenar.
   - $\Sigma_c^0$ se puede inicializar con la matriz identidad, o como $R^T R$ donde $R$ es una matriz de números.

In [6]:
import numpy as np

def isPSD(A, tol=1e-8):
  E = np.linalg.eigvalsh(A)
  return np.all(E > -tol)

def init_params(X, k):
    """
    Centroids initialization for K-Means algorithm.
    :param X: Data.
    :param int k: Number of clusters.
    :return: Randomly chosen parameters.
    """
    # Dimension
    N, d = X.shape
    
    # Choose k random positive numbers between 0 and 1 that sum 1 (Hint: Use np.random.dirichlet(alpha=(1,) * k))
    weights = np.random.dirichlet(alpha=(1,) * k)
    # Choose k random points from X (Hint: Use np.random.randint(N, size=k)
    means = np.array([X[i] for i in np.random.randint(N, size=k)])
    # Choose k random positive definite matrices 
    # (Hint: Use np.random.rand(d, d) to generate a random matrix of d x d)
    covariances = np.zeros((k, d, d))
    for c in range(k):
        condition = True
        while condition == True:
            R = np.triu(np.random.rand(d, d), k=0)
            i_lower = np.tril_indices(d, -1)
            R[i_lower] = R.T[i_lower]
            condition = isPSD(R,1e-8)
        min_eig = np.min(np.real(np.linalg.eigvals(R)))
        if min_eig < 0:
            R -= 10 * min_eig * np.eye(*R.shape)
        covariances[c, :, :] = covariances[c, :, :] + R
    
    # The weights must be an array of size k
    assert weights.shape == (k,)
    # The means must be a (k, d) array
    assert means.shape == (k, d)
    # The weights must be an (k, d, d) array
    assert covariances.shape == (k, d, d)
    
    return weights, means, covariances

In [7]:
X = np.array([[1,2,3,4,5],[6,7,8,9,10],[1,1,3,4,1],[6,7,8,2,3]])
weights, means, covariances = init_params(X, 2)
print("-------------------------weights-------------------------")
print(weights)
print("-------------------------means-------------------------")
print(means)
print("-------------------------covs-------------------------")
print(covariances)

-------------------------weights-------------------------
[0.60272109 0.39727891]
-------------------------means-------------------------
[[ 6  7  8  9 10]
 [ 1  1  3  4  1]]
-------------------------covs-------------------------
[[[9.21915177 0.18630776 0.3231478  0.72034688 0.19337768]
  [0.18630776 9.19608178 0.36345851 0.4450577  0.9983745 ]
  [0.3231478  0.36345851 9.8349224  0.83985971 0.63167597]
  [0.72034688 0.4450577  0.83985971 9.25748364 0.88838163]
  [0.19337768 0.9983745  0.63167597 0.88838163 9.75597946]]

 [[9.47695902 0.0151853  0.73011257 0.42759638 0.90536347]
  [0.0151853  8.98631429 0.78639269 0.73435126 0.54012184]
  [0.73011257 0.78639269 9.72540404 0.42294038 0.66184315]
  [0.42759638 0.73435126 0.42294038 8.9517617  0.28110688]
  [0.90536347 0.54012184 0.66184315 0.28110688 9.39912854]]]


### 2. E-step

En este paso, debemos calcular la distribución variacional que maximiza la cota variacional con:

$$
q(t_i=c) = p(t_i=c|x_i, \theta) = \frac{\mathcal{N}(x_i | \mu_c, \Sigma_c) \pi_c}{\sum_{c=1}^{k} \pi_c \mathcal{N}(x | \mu_c, \Sigma_c)} = \frac{\mathcal{N}(x_i | \mu_c, \Sigma_c) \pi_c}{Z},
$$

donde el denominador $Z$ es simplemente un número que normaliza el numerador para que se satisfaga que $\sum_{c=1}^{k} q(t_i=c) = 1$.

En este sentido una manera de implementar lo anterior es:

1. Calcular:

    $$
    \tilde{q}(t_i=c) = \pi_c \mathcal{N}(x_i | \mu_c, \Sigma_c).
    $$

2. Normalizar:

    $$
    q(t_i = c) = \frac{\tilde{q}(t_i=c)}{\sum_{c=1}^{k} \tilde{q}(t_i=c)}.
    $$

In [8]:
from scipy.stats import multivariate_normal
def e_step_gmm(X, weights, means, covariances):
    """
    Performs E-step on GMM model.
    :param X: Data.
    :param weights: mixture component weights pi
    :param means: mixture component means mu
    :param covariances: mixture component covariance matrices sigma
    :return: Variational distribution q.
    """
    # Dimensions
    N, d = X.shape
    k = weights.shape[0]
    
    # Pre-alloc of variational distribution q(t)
    q = np.zeros((N, k))
    q_tilde = np.zeros((N, k))
    # Calculate unnormalized distribution 
    # (Hint: Use multivariate_normal.pdf(X, mean=..., cov=...) for the likelihood N(x | mu, sigma))
    for c in range(k):
        likelihood = multivariate_normal.pdf(X, mean=means[c], cov=covariances[c,:,:])
        q_tilde[:, c] = weights[c] * likelihood
    
    # Normalize
    q = q_tilde / q_tilde.sum(axis=1).reshape(-1, 1)
    
    # The variational distribution q must be a (k, d) array
    assert q.shape == (N, k)
    
    return q

In [9]:
q = e_step_gmm(X, weights, means, covariances)
print(q)
print(q.shape)

[[1.64154975e-02 9.83584502e-01]
 [9.99859123e-01 1.40876715e-04]
 [3.82700814e-04 9.99617299e-01]
 [5.60781529e-01 4.39218471e-01]]
(4, 2)


### 3. M-step

En este paso calculamos los parámetros para maximizar 

$$
\max_{\theta} \sum_{i=1}^{N} \mathbb{E}_{q(t_i)} \left[\log p(x_i, t_i| \theta)\right] = \sum_{i=1}^{N} \sum_{c=1}^{k} q(t_i=c) \log \left[\mathcal{N}(x_i | \mu_c, \Sigma_c) \pi_c\right].
$$

Esto resulta en:

$$
\mu_c = \frac{\sum_{i=1}^{N} q(t_i=c)x_i}{\sum_{i=1}^{N} q(t_i=c)},
$$

$$
\Sigma_c = \frac{\sum_{i=1}^{N} q(t_i=c)(x_i - \mu_c) (x_i - \mu_c)^T}{\sum_{i=1}^{N} q(t_i=c)},
$$

$$
\pi_c = \frac{\sum_{i=1}^{N} q(t_i=c)}{N}.
$$

In [10]:
def m_step_gmm(X, q):
    """
    Performs M-step on GMM model.
    :param X: Data.
    :param q: Variational distribution q.
    :return: Next iteration of parameters weights, means, covariances.
    """
    # Dimensions
    N, d = X.shape
    k = q.shape[1]

    # Means
    means = np.sum(np.array([[(q[i][c] * X[i]) / q[:][c].sum() for i in range(N)] for c in range(k)]),axis=1)
    # Covariances
    covariances = np.zeros((k, d, d))
    for c in range(k):
        aux = np.sum(np.array([(q[i][c] * np.matmul((((X[i] - means[c]).reshape(1,d)).T),(X[i] - means[c]).reshape(1,d))) / q[:][c].sum() for i in range(N)]),axis=0)
        covariances[c, :, :] = aux[np.newaxis,:,:]
    # Weights
    weights = np.array([q[:][c].sum() / N for c in range(k)])
    
    # The weights must be an array of size k
    assert weights.shape == (k,)
    # The means must be a (k, d) array
    assert means.shape == (k, d)
    # The weights must be an (k, d, d) array
    assert covariances.shape == (k, d, d)
    
    return weights, means, covariances

In [11]:
m_step_gmm(X, q)

(array([0.25, 0.25]),
 array([[ 9.38064211, 10.95769826, 12.53551981, 10.18748796, 11.76339601],
        [ 4.61935789,  6.04230174,  9.46448019,  8.81251204,  7.23660399]]),
 array([[[19.01598213, 22.14495669, 25.27166268, 20.40683209,
          23.5392095 ],
         [22.14495669, 25.80003298, 29.45233065, 23.80382018,
          27.46306443],
         [25.27166268, 29.45233065, 33.63124047, 27.20057494,
          31.38388013],
         [20.40683209, 23.80382018, 27.20057494, 39.645015  ,
          43.04235308],
         [23.5392095 , 27.46306443, 31.38388013, 43.04235308,
          46.97076683]],
 
        [[26.8169456 , 33.21421734, 45.51315201, 30.4127176 ,
          27.95749496],
         [33.21421734, 41.89002174, 57.66959042, 40.52555463,
          38.54571261],
         [45.51315201, 57.66959042, 83.81931478, 66.08013027,
          57.24663973],
         [30.4127176 , 40.52555463, 66.08013027, 66.31576828,
          53.26599741],
         [27.95749496, 38.54571261, 57.24663973, 

### 4. Calcular log-verosimilitud

Finalmente, necesitamos una función para dar seguimiento a la convergencia del algoritmo. Pueden haber varias selecciones para esta función:

1. La cota variacional.
2. La log-verosimilitud.

Lo que haremos es que pararemos la iteraciones cuando alguna de estas funciones no varíe significativamente. 

Otra utilidad de esta función es que nos sirve para verificar que el algoritmo esté funcionando correctamente. Debemos recordar que este algoritmo nos asegura que esta función no puede decrecer. Si lo hace es porque algo estamos haciendo mal.

Programemos la log-verosimilitud (promediada por el número de puntos):

$$
g(\theta) := \frac{1}{N}\log p(X | \theta) = \frac{1}{N}\sum_{i=1}^{N} \log \left( \sum_{c=1}^{k} \pi_c \mathcal{N}(x_i | \mu_c, \Sigma_c) \right).
$$

**Esta ya la hicimos en clase, no hay necesidad de que la vuelvan a hacer**

In [205]:
def log_likelihood_gmm(X, weights, means, covariances):
    """
    Log-likelihood of the data wrt Gaussian Mixture Model.
    :param data: Data.
    :param mu: Means of the components of the GMM.
    :param sigma: Covariances of the components of the GMM.
    :param pi: Weights of the components of the GMM.
    :return: Log-likelihood of the data wrt GMM.
    """
    # Number of components
    k = means.shape[0]
    # Number of points
    N = X.shape[0]
    
    # Individual likelihood of each point to each normal
    ind_likelihood = np.zeros((N, k))
    for c in range(k):
        ind_likelihood[:, c] = multivariate_normal.pdf(X, mean=mumeans[c, :], cov=covariances[c, :, :])
    
    # Log likelihood
    log_likelihood = np.log(ind_likelihood.dot(weights)).mean()
        
    return log_likelihood

### 5. Juntemos todo

Ya tenemos todos los componentes para nuestro modelo de mezclas Gaussianas. Haremos lo siguiente:

1. Inicializaremos los parámetros $\pi$, $\mu$, y $\Sigma$.
2. Iteraremos hasta que:
   - la log-verosimilitud no cambie significativamente, es decir hasta que: $\left\lvert \frac{g(\theta^{j}) - g(\theta^{j-1})}{g(\theta^{j-1})}\right\rvert \leq \epsilon$, donde $\epsilon$ es una tolerancia para la convergencia (usualmente $\epsilon = 10^{-3}$ o menor).
   - Alternativamente, iteraremos un máximo número de veces. Esto para prevenir que el algoritmo se quede iterando por mucho tiempo.
3. Este proceso lo repetiremos varias veces, esto debido a que el algoritmo de maximización de la esperanza solo alcanza máximos locales, entonces intentaremos con varias inicializaciones y nos quedaremos con la mejor.

In [206]:
from math import inf
def gmm(X, k, eps=1e-3, max_iter=100, restarts=50):
    """
    Starts with random initialization *restarts* times. Runs optimization until saturation with *eps* reached
    or *max_iter* iterations were made.
    :param X: Data.
    :param k: Number of components.
    :param eps: Tolerance for convergence.
    :param max_iter: Maximum number of iterations at each run.
    :param restarts: Number of restarts.
    :return: Array of
    """
    # Dimensions
    N, d = X.shape # number of objects
    
    best_loss = -inf
    best_weights = None
    best_means = None
    best_covariances = None

    for _ in range(restarts):
        try:
            # Parameters initialization
            weights, means, covariances = init_params(X, k)
            loss = [-inf]
            for _ in range(max_iter):
                try:
                    # E-step
                    q = e_step_gmm(X, weights, means, covariances)
                    # M-step
                    weights, means, covariances = m_step_gmm(X, q)
                    # Log-likelihood
                    loss.append(log_likelihood_gmm(X, weights, means, covariances))
                    # Check for convergence
                    if np.abs((loss[-1] - loss[-2]) / loss[-2]) <= eps:
                        # Keep the best params
                        if loss[-1] > best_loss:
                            best_loss = loss[-1]
                            best_weights = weights
                            best_means = means
                            best_covariances = covariances
                        break
                except ValueError:
                    pass

        except np.linalg.LinAlgError:
            print("Singular matrix: components collapsed")
            pass

    return best_loss, best_weights, best_means, best_covariances

In [2]:
pip install bank_customer_data

Defaulting to user installation because normal site-packages is not writeable


ERROR: Could not find a version that satisfies the requirement bank_customer_data (from versions: none)
ERROR: No matching distribution found for bank_customer_data


You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


Ahora, probemos nuestro algoritmo:

In [3]:
# Importamos función para generar datos
from bank_customer_data import generate_bank_customer_data
# Importamos pyplot
from matplotlib import pyplot as plt

ModuleNotFoundError: No module named 'bank_customer_data'

In [208]:
# Generamos datos
data = generate_bank_customer_data()

NameError: name 'generate_bank_customer_data' is not defined

In [0]:
best_loss, best_weights, best_means, best_covariances = gmm(data[["income", "debt"]].to_numpy(), k=3)

In [0]:
# Gaussianas ajustadas
X1 = multivariate_normal(mean=best_means[0, :], cov=best_covariances[0, :, :])
X2 = multivariate_normal(mean=best_means[1, :], cov=best_covariances[1, :, :])
X3 = multivariate_normal(mean=best_means[2, :], cov=best_covariances[2, :, :])

In [0]:
# Datos
plt.scatter(data["income"], data["debt"], c="gray", alpha=0.6)

# Gaussiana 1
x = np.linspace(0, 10, 100)
y = np.linspace(0, 8, 100)
x, y = np.meshgrid(x, y)
z = X1.pdf(np.dstack([x, y]))
plt.contour(x, y, z)

# Gaussiana 2
x = np.linspace(0, 10, 100)
y = np.linspace(0, 8, 100)
x, y = np.meshgrid(x, y)
z = X2.pdf(np.dstack([x, y]))
plt.contour(x, y, z)

# Gaussiana 3
x = np.linspace(0, 10, 100)
y = np.linspace(0, 8, 100)
x, y = np.meshgrid(x, y)
z = X3.pdf(np.dstack([x, y]))
plt.contour(x, y, z)

plt.xlabel("Ingresos mensuales (x100k MXN)")
plt.ylabel("Deuda (x100k MXN)")

In [0]:
# Por colores
plt.scatter(data["income"], data["debt"], c=gmm.predict_proba(X), alpha=0.6)
plt.xlabel("Ingresos mensuales (x100k MXN)")
plt.ylabel("Deuda (x100k MXN)")

<script>
  $(document).ready(function(){
    $('div.prompt').hide();
    $('div.back-to-top').hide();
    $('nav#menubar').hide();
    $('.breadcrumb').hide();
    $('.hidden-print').hide();
  });
</script>

<footer id="attribution" style="float:right; color:#808080; background:#fff;">
Created with Jupyter by Esteban Jiménez Rodríguez.
</footer>